In [ ]:
!pip install pymysql sqlalchemy

In [313]:
import csv,re
import pandas as pd
import logging
from sqlalchemy import create_engine


In [334]:
# read csv use pandas
df = pd.read_csv('top_1000_instagram.csv')

,Name,Rank,Category,Followers,Audience Country,Authentic Engagement,Engagement Avg.
0,cristiano,1,Sports with a ball,462.9M,India,5.5M,6.6M
1,leomessi,2,Sports with a ballFamily,347.2M,Argentina,3.6M,4.8M
2,kendalljenner,3,ModelingFashion,247.6M,United States,3M,4.9M
3,arianagrande,4,Music,321.4M,United States,2.4M,3.4M
4,zendaya,5,Cinema & Actors/actressesFashion,147M,United States,4.3M,5.8M
...,...,...,...,...,...,...,...
994,garciajoshuae,995,Modeling,5M,Philippines,357.5K,412.1K
995,nimratkhairaofficial,996,Music,8M,India,211.1K,261.8K
996,hanhyojoo222,997,TravelCinema & Actors/actresses,4M,Indonesia,434.2K,518.2K
997,memphisdepay,998,Sports with a ball,15M,France,76.4K,135.5K


In [315]:
# function transfrom million
def transform_million(df):
    clean = df.lower()
    return re.sub(r"[?\[m\]|\[k\]]","",clean)


In [335]:
# function transfrom data,filtering,cleaning data and mapping column
def transform(df):
    logging.info("Transfrom data in processing")

    mapping = {
        "Name" : "nama",
        "Rank" : "rank",
        'Category':'kategori',
        "Followers":"followers",
        "Audience Country":"audiens_negara",
        "Authentic Engagement":"authentic_engagement",
        "Engagement Avg.":"avg_engagement",
    }
    rename_df = df.rename(columns=mapping)
    rename_df["authentic_engagement_dalam_juta"] =rename_df["authentic_engagement"].apply(lambda value : (transform_million(value)))
    rename_df["followers_dalam_juta"] =rename_df["followers"].apply(lambda value : (transform_million(value)))
    rename_df["avg_engagement_dalam_juta"] =rename_df["avg_engagement"].apply(lambda value : (transform_million(value)))
    df = rename_df[["nama","rank",'kategori','followers_dalam_juta','audiens_negara','authentic_engagement_dalam_juta','avg_engagement_dalam_juta']].dropna()
    res_df =  df[(df.audiens_negara == "Indonesia")].reset_index(drop=True)
    res_df['rank'] = res_df.index+1
    return res_df
df2 = transform(df)
df2


,nama,rank,kategori,followers_dalam_juta,audiens_negara,authentic_engagement_dalam_juta,avg_engagement_dalam_juta
0,lalalalisa_m,1,Music,79.9,Indonesia,6.3,7.3
1,jennierubyjane,2,Music,68.2,Indonesia,7,8.3
2,sooyaaa__,3,MusicBeauty,62.1,Indonesia,4.9,5.6
3,roses_are_rosie,4,Music,61,Indonesia,4,4.6
4,bts.bighitofficial,5,Music,66,Indonesia,3,3.3
...,...,...,...,...,...,...,...
106,valeyellow46,107,Racing Sports,13.9,Indonesia,119.3,155.9
107,hitocaesar,108,Lifestyle,13.6,Indonesia,112.9,159
108,wayvofficial,109,Music,7.3,Indonesia,238.2,283.7
109,thariqhalilintar,110,Shows,7.4,Indonesia,225.6,279.5


In [317]:
# connector database mysql
DB_NAME = "dataset"
DB_USER = "root"
DB_PASSWORD = ""
DB_HOST = "localhost"
DB_PORT = "3306"
CONNECTING = f"mysql+pymysql://{DB_USER}{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?charset=utf8mb4"
TABLE_NAME = "top_1000_instagram"


In [318]:
# write dataframe to database mysql
def write_dbs(df,db_name,table_name,connecting_string):
    engine = create_engine(connecting_string)
    logging.info(f"Writing dataframe to database: '{db_name}', table: {table_name}...")
    df.to_sql(name = table_name,con=engine,if_exists="replace",index=False)

In [319]:
write_dbs(df =df2,db_name=DB_NAME,table_name=TABLE_NAME,connecting_string=CONNECTING)

In [320]:
# function for read data from mysql database
def read_from_mysql(db_name,table_name,connection_string):
    engine = create_engine(connection_string)
    logging.info(f"Reading data from mysql database : '{db_name}', table: {table_name}") 
    return pd.read_sql(table_name,con=connection_string)

In [321]:
# RESULT
ress = read_from_mysql(db_name=DB_NAME,table_name=TABLE_NAME,connection_string=CONNECTING)
ress

,nama,rank,kategori,followers_dalam_juta,audiens_negara,authentic_engagement_dalam_juta,avg_engagement_dalam_juta
0,lalalalisa_m,1,Music,79.9,Indonesia,6.3,7.3
1,jennierubyjane,2,Music,68.2,Indonesia,7,8.3
2,sooyaaa__,3,MusicBeauty,62.1,Indonesia,4.9,5.6
3,roses_are_rosie,4,Music,61,Indonesia,4,4.6
4,bts.bighitofficial,5,Music,66,Indonesia,3,3.3
...,...,...,...,...,...,...,...
106,valeyellow46,107,Racing Sports,13.9,Indonesia,119.3,155.9
107,hitocaesar,108,Lifestyle,13.6,Indonesia,112.9,159
108,wayvofficial,109,Music,7.3,Indonesia,238.2,283.7
109,thariqhalilintar,110,Shows,7.4,Indonesia,225.6,279.5
